In [6]:
from QLearningAgent import QLearningAgent
from PPORewardRecall import RewardCallback
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import stable_baselines3
import gymnasium as gym

from FrozenLakeRewardWrapper import FrozenLakeRewardWrapper
from PPOAgent import PPOAgent
from stable_baselines3 import PPO

# 创建 FrozenLake 环境 (Q-Learning)
map = ['SFHFFFFHFHFFFFFF', 'FFFHFFFFFFFFHFFF', 'FFFFHFFFFHFFHFHF', 'FFFFFFFFFFFFFFFF', 'FFHFFFFFFFFFFHFF', 'FFFHFFHFHFFFHFHF', 'HFFFHHFFHFFFFHHF', 'FFFFHFFFFFFFHFFF', 'FHFFFFFFFFFFFHFF', 'FHFFFFFFFFHFFFHH', 'FHFFFFFFFFHFHFFF', 'HFHFFHHFFFFFHFFF', 'FFFFFFFFFFFFHFFF', 'HFFFHFFHFFFFFFFF', 'FFFHFFFFFHFFFFFF', 'FFFHFFFFFFFHFFFG']#generate_random_map(size=16)

In [7]:
training_env1 = gym.make('FrozenLake-v1',desc=map, is_slippery = False)# map_name='4x4', is_slippery=True)#
wrapped_training_env = FrozenLakeRewardWrapper(training_env1, step_penalty=0.02)

eval_env = gym.make('FrozenLake-v1',desc=map, is_slippery = False)#map_name='4x4', is_slippery=True)#
wrapped_eval_env = FrozenLakeRewardWrapper(eval_env, step_penalty=0.02)

agent = QLearningAgent(wrapped_training_env)
model = PPO(
    policy="MlpPolicy",            # 使用多层感知机策略
    env=wrapped_training_env,      # 传入包装后的训练环境
    verbose=0,                     # 显示训练过程的详细信息
    learning_rate=5e-4,            # 设置学习率
    clip_range=0.3,                # 设置剪切范围（epsilon_clip）
    n_epochs=3,                    # 训练周期数
    batch_size=128,                 # 批次大小
    gamma=0.99,                    # 折扣因子
    gae_lambda=0.95,               # GAE 的 lambda 参数
    device='cpu',
    policy_kwargs=dict(net_arch=[64, 64, 64]),
    ent_coef= 0.02
)
reward_callback = RewardCallback(
    eval_env=wrapped_eval_env,
    eval_freq=5000,
    n_eval_episodes=100,
    verbose=1
)

'''
print("Start training...")
agent.train(episodes=300000)
'''
model.learn(
    total_timesteps=100000,
    callback=reward_callback
)
wrapped_training_env.close()

Step 5000	平均奖励: -11.90 +/- 0.00
Step 10000	平均奖励: -2.10 +/- 0.00
Step 15000	平均奖励: -2.10 +/- 0.00
Step 20000	平均奖励: -2.10 +/- 0.00
Step 25000	平均奖励: -2.10 +/- 0.00
Step 30000	平均奖励: -2.10 +/- 0.00
Step 35000	平均奖励: -2.10 +/- 0.00
Step 40000	平均奖励: -2.10 +/- 0.00
Step 45000	平均奖励: -2.10 +/- 0.00
Step 50000	平均奖励: -2.10 +/- 0.00
Step 55000	平均奖励: -2.10 +/- 0.00
Step 60000	平均奖励: -2.10 +/- 0.00
Step 65000	平均奖励: -2.10 +/- 0.00
Step 70000	平均奖励: -2.10 +/- 0.00
Step 75000	平均奖励: -2.10 +/- 0.00
Step 80000	平均奖励: -2.10 +/- 0.00
Step 85000	平均奖励: -2.10 +/- 0.00
Step 90000	平均奖励: -2.10 +/- 0.00
Step 95000	平均奖励: -2.10 +/- 0.00
Step 100000	平均奖励: -2.10 +/- 0.00


In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

'''
agent.env = wrapped_eval_env
print("Start evaluation...")
agent.evaluate(episodes=100)
'''
mean_reward, std_reward = evaluate_policy(
    model,
    wrapped_eval_env,
    n_eval_episodes=100,
    deterministic=True,  # 使用确定性策略进行评估
    render=False
)
print(f"Final Evaluation\t平均奖励: {mean_reward:.2f} +/- {std_reward:.2f}")

Final Evaluation	平均奖励: -2.10 +/- 0.00


In [17]:

import gymnasium as gym
final_env = gym.make('FrozenLake-v1', is_slippery=False, render_mode = 'human', desc = map)#, map_name='4x4')#
wrapped_final_env = FrozenLakeRewardWrapper(final_env, step_penalty=0.02)

#agent.env = wrapped_final_env
model.set_env(wrapped_final_env)

# 重置环境，获取初始状态
state, info = wrapped_final_env.reset()

done = False  # 游戏是否结束
step_count = 0  # 计步

while not done:
    wrapped_final_env.render()  # 渲染环境
    #action = agent.choose_best_action(state)  # 选择动作
    #state, reward, terminated, truncated, _ = wrapped_final_env.step(action)  # 执行动作
    action, _ = model.predict(state, deterministic=True)
    state, reward, terminated, truncated, _ = wrapped_final_env.step(action[()])  # 执行动作
    done = terminated or truncated
    print(f"Step {step_count}: Action={action}, State={state}, Reward={reward}, Done={done}")
    step_count += 1

# 游戏结束后关闭环境
wrapped_final_env.close()


Step 0: Action=2, State=1, Reward=-0.1, Done=False
Step 1: Action=2, State=2, Reward=-2.0, Done=True


In [10]:
print(map)

['SFHFFFFHFHFFFFFF', 'FFFHFFFFFFFFHFFF', 'FFFFHFFFFHFFHFHF', 'FFFFFFFFFFFFFFFF', 'FFHFFFFFFFFFFHFF', 'FFFHFFHFHFFFHFHF', 'HFFFHHFFHFFFFHHF', 'FFFFHFFFFFFFHFFF', 'FHFFFFFFFFFFFHFF', 'FHFFFFFFFFHFFFHH', 'FHFFFFFFFFHFHFFF', 'HFHFFHHFFFFFHFFF', 'FFFFFFFFFFFFHFFF', 'HFFFHFFHFFFFFFFF', 'FFFHFFFFFHFFFFFF', 'FFFHFFFFFFFHFFFG']
